# Spectral Analysis for Modal Parameter Linear Estimate

## Setup


### Libraries

In [ ]:
# Install requirements
from IPython.display import clear_output
import sys
!$sys.executable -m pip install lim-sample[notebooks]==0.0.2
clear_output()

### Generate test audio

In [ ]:
from matplotlib import pyplot as plt
from librosa.display import waveplot, specshow
from IPython.display import Audio as play
from sample.utils import test_audio
import numpy as np

def resize(w=12, h=6):
  plt.gcf().set_size_inches([w, h])

fs = 44100
x = test_audio(fs=fs)

waveplot(x, sr=fs, alpha=.5, zorder=100)
plt.grid()
resize()
play(x, rate=fs)

## Sinusoidal Model

In [ ]:
from sample import SAMPLE
sample = SAMPLE(
    sinusoidal_model__max_n_sines=10,
    sinusoidal_model__peak_threshold=-30,
    sinusoidal_model__save_intermediate=True
).fit(x)

### STFT

In [ ]:
stft = np.array([
  mx
  for mx, _ in sample.sinusoidal_model.intermediate_["stft"]
]).T

specshow(stft, sr=fs, x_axis="time", y_axis="hz");
plt.ylim([0, 2000])
resize()

### Peak detection

In [ ]:
mx, px = sample.sinusoidal_model.intermediate_["stft"][0]
f = fs * np.arange(mx.size) / sample.sinusoidal_model.w_.size
ploc, pmag, pph = sample.sinusoidal_model.intermediate_["peaks"][0]

ax = plt.subplot(121)
plt.fill_between(f, np.full(mx.shape, -120), mx, alpha=.1)
plt.plot(f, mx)
plt.scatter(ploc * fs / sample.sinusoidal_model.w_.size, pmag, c="C0")
plt.ylim([-60, plt.ylim()[1]])
plt.grid()
plt.title("magnitude")

plt.subplot(122, sharex=ax)
plt.plot(f, px)
plt.scatter(ploc * fs / sample.sinusoidal_model.w_.size, pph)
plt.ylim([np.min(px[f < 2000]), np.max(px[f < 2000])])
plt.grid()
plt.title("phase")
plt.xlim([0, 2000])
resize()

### Sine tracking

In [ ]:
from sample import plots
plots.sine_tracking_2d(sample.sinusoidal_model)
resize()

In [ ]:
from sample import plots
plots.sine_tracking_3d(sample.sinusoidal_model)
resize(6, 6)

## Regression

In [ ]:
_, ax = plt.subplots(1, 2, sharex=True, sharey=True)
plots.sine_tracking_2d(sample.sinusoidal_model, ax=ax)

t_x = np.arange(x.size) / fs
ax[0].cla()
for f, d, a in sample.param_matrix_.T:
    ax[0].plot(t_x, 20*np.log10(a * np.exp(-2*t_x / d)))

ax[0].set_title("fitted curves")
ax[0].grid()
ax[0].set_ylabel("magnitude (dB)")
ax[0].set_xlabel(ax[1].get_xlabel())
ax[1].set_title("ground truth")
ax[1].set_ylabel("")
resize()

# Resynthesize

In [ ]:
x_hat = sample.predict(np.arange(x.size) / fs)

waveplot(x_hat, sr=fs, alpha=.5, zorder=100)
plt.grid()
resize()
play(x_hat, rate=fs)

Play against each other in stereo

In [ ]:
x_dual = np.array([x, x_hat])
for l, xi in zip(("original", "resynthesis"), x_dual):
    waveplot(xi, sr=fs, alpha=.5, zorder=100, label=l)
plt.grid()
plt.legend()
resize()
play(x_dual, rate=fs)